In [12]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from linetools.spectra import utils as ltsu
from linetools.spectra.xspectrum1d import XSpectrum1D
from astropy.io import fits

In [2]:
import clamato_read
import cuts
import milkyway
import prep

In [3]:
#grab the spectra
spec, red, coord = clamato_read.xspec([2.25,2.75])

#cut the sample
spec, red, coord = cuts.lyman_limit(spec, red, coord)
spec, red, coord = cuts.s2n(spec, red, coord)

#deredden
spec = milkyway.extinction(spec, red, coord)

/opt/anaconda3/lib/python3.7/site-packages/astropy/units/quantity.py:464: RuntimeWarning: invalid value encountered in greater
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


Number of spectra in the redshift bin: 416
Number of spectra provided = 416
Number of spectra left after the lyman-limit cut =  367
Number of spectra provided = 367
Number of spectra left after the S/N cut =  281
Loading pixel_info ...
Loading samples ...
Loading best_fit ...
Replacing NaNs in reliable distance estimates ...
Sorting pixel_info ...
Extracting hp_idx_sorted and data_idx at each nside ...
  nside = 64
  nside = 128
  nside = 256
  nside = 512
  nside = 1024
t = 62.787 s
  pix_info:   0.663 s
   samples:  40.632 s
      best:   8.146 s
       nan:   1.666 s
      sort:  11.263 s
       idx:   0.417 s


/Users/jsmonzon/linetools/linetools/spectra/xspectrum1d.py:253: UserWarning: No unit given to wavelength, assuming Angstroms.
  warnings.warn("No unit given to wavelength, assuming Angstroms.")


In [4]:
#creating the two bins
lowz_cut = tuple([red < 2.5])
hiz_cut = tuple([red > 2.5])

low_spec, low_red = spec[lowz_cut], red[lowz_cut]
hi_spec, hi_red = spec[hiz_cut], red[hiz_cut]

In [5]:
low_rest = prep.clean(low_spec, low_red)
hi_rest = prep.clean(hi_spec, hi_red)

In [25]:
def stacks(spec, red, N, zbin, plot=False):
    """
    :param spec: XSpectrum1D collated object :rest_spec from the stacking modules
    :red: redshift array
    :param N: number of iterations for the bootstrap
    :param zbinn: which bin is being used (str) "lowz" or "hiz"
    """

    import numpy as np
    from astropy.io import fits
    from numpy import random as ran
    from linetools.spectra import utils as ltsu
    from linetools.spectra.xspectrum1d import XSpectrum1D

    print("Number of spectra provided =", spec.nspec)

    # the actual stack
    stack = ltsu.smash_spectra(spec)
    z_med = np.median(red)

    # the bootstrap
    R = (spec.nspec)  # restraints on the random variable
    N_stack = []
    N_z_med = []

    for i in np.arange(N):  # this might take a while
        
        if i%10 == 0:
            print(i)
        choice = np.asarray(ran.randint(0, R, R))  # the shuffle

        rand_spec = np.asarray([spec[index] for index in choice])
        rand_red = np.asarray([red[index] for index in choice])

        rand_collate = ltsu.collate(rand_spec)  # collate and stack
        N_stack.append(ltsu.smash_spectra(rand_collate))
        N_z_med.append(np.median(rand_red))
        
    # matrix math to create the error array
    N_flux = np.array([entry.flux for entry in N_stack])
    N_matrix = np.array([N_flux[i] - stack.flux for i in range(N)])

    tranpose = np.transpose(N_matrix)  # matrix math
    covariance = np.dot(tranpose, N_matrix)
    sigma = np.sqrt(np.diagonal(covariance) / (N - 1))  # the final error array

    composite = XSpectrum1D(stack.wavelength, stack.flux, sig=sigma)
    #plot with bootstrap generated error
    if plot==True:
        composite.plot()
    
    #writing out the stack
    hdr = fits.Header()
    hdr["REDSHIFT"] = z_med
    hdr["NSPEC"] = spec.nspec
    header = fits.PrimaryHDU(header=hdr)

    c_wave = fits.Column(name='WAVELENGTH', array=stack.wavelength, unit="angstroms", format="E") 
    c_flux = fits.Column(name='FLUX', array=stack.flux, unit="relative flux", format="E")
    c_noise = fits.Column(name='NOISE', array=sigma, unit="relative flux", format="E")
    table = fits.BinTableHDU.from_columns([c_wave, c_flux, c_noise])
    
    full_hdu = fits.HDUList([header, table])
    full_hdu.writeto("../../fits/composites/"+zbin+"/composite.fits")
    
    #writing out the boostrap
    hdr = fits.Header()
    hdr["ZBIN"] = zbin
    hdr["NBOOT"] = N
    header = fits.PrimaryHDU(header=hdr)
    
    c_wave = fits.Column(name='WAVELENGTH', array=stack.wavelength, unit="angstroms", format="E") 
    c_noise = fits.Column(name='FLUX_ERR', array=sigma, unit="relative flux", format="E")
    table = fits.BinTableHDU.from_columns([c_wave, c_noise])
    
    flux_data = fits.ImageHDU(N_flux, name="FLUX_ITERATIONS")
    
    full_hdu = fits.HDUList([header, table, flux_data])
    full_hdu.writeto("../../fits/bootstrap/"+zbin+"/stacks.fits")

In [26]:
stacks(hi_rest,hi_red,100,"hiz")

Number of spectra provided = 142
0
10
20
30
40
50
60
70
80
90


/Users/jsmonzon/linetools/linetools/spectra/xspectrum1d.py:253: UserWarning: No unit given to wavelength, assuming Angstroms.
  warnings.warn("No unit given to wavelength, assuming Angstroms.")


In [35]:
stack = fits.open("../../fits/composites/hiz/composite.fits")

In [38]:
stack[1].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   12 / length of dimension 1                          
NAXIS2  =                  372 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    3 / number of table fields                         
TTYPE1  = 'WAVELENGTH'                                                          
TFORM1  = 'E       '                                                            
TUNIT1  = 'angstroms'                                                           
TTYPE2  = 'FLUX    '                                                            
TFORM2  = 'E       '        